# Average out different circuit permutations for any cost-GC. Runs should be around circuit generation as well to capture different permutations of the circuit.

In [9]:
from qiskit import *
import pandas as pd
from qiskit import transpile
import random
import csv
import numpy as np
from qiskit.providers.aer import AerSimulator
import time
import itertools
import random
import progressbar
import pathlib
from IPython.display import clear_output
from qiskit.tools.monitor import job_monitor

from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
provider = IBMQ.load_account()
# IBMQ.load_account()
provider = IBMQ.get_provider(hub = 'ibm-q-research-2', group = 'los-gatos-hs-1', project = 'main')
backend = provider.get_backend('ibm_perth')

qubitCount = 7
runs = 1

noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

Gates = ['x', 'y', 'z', 'h', 'cx', 'swap']
gateCosts = [1, 1, 1, 2, 5, 11]
fields = ['cost', 'gate count', 'tqc3TT', 'tqc3RT', 'tqc3GC', 'tqc2TT', 'tqc2RT', 'tqc2GC', 'tqc1TT', 'tqc1RT', 'tqc1GC', 'tqc0TT', 'tqc0RT', 'tqc0GC']
# ['cost', 'depth', 'tqc3TT', 'tqc3RT', 'tqc3Counts', 'tqc3Depth', 'tqc2TT', 'tqc2RT', 'tqc2Counts', 'tqc2Depth', 'tqc1TT', 'tqc1RT', 'tqc1Counts', 'tqc1Depth', 'tqc0TT', 'tqc0RT', 'tqc0Counts', 'tqc0Depth', 'sim']

ibmqfactory.load_account:WARNING:2023-01-17 18:39:01,018: Credentials are already in use. The existing account in the session will be replaced.
C:\Users\taren\anaconda3\lib\site-packages\qiskit\providers\aer\noise\device\models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (183398.64641532142 > 2 * 47915.39285548773). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
C:\Users\taren\anaconda3\lib\site-packages\qiskit\providers\aer\noise\device\models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (183398.64641532142 > 2 * 47915.39285548773). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
C:\Users\taren\anaconda3\lib\site-packages\qiskit\providers\aer\noise\device\models.py:264: UserWarning: Device model returned an invalid 

In [10]:
def QCtoDF(qc):
    string = qc.qasm()
    circuit = string.split(';')
    #Changed to 3: from 4:
    circuit = circuit[3:]
    circuit.pop(len(circuit)-1)
    #print(circuit)
    with open(r"gatesTemp.csv", 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(['Gate', 'Qubit'])
        for i in range(len(circuit)):            
            circuit[i] = circuit[i].replace("\n", '')
            #print(circuit[0][1])
            temp = circuit[i].split(' ')
            #temp[1] = temp[1].split(',')
            for j in range(len(temp[1])):
                 temp[1] = temp[1].replace("q[", '')
                 temp[1] = temp[1].replace("]", '')
            if temp[0] != 'measure' and temp[0] != 'barrier' and temp[0] != 'creg':
                writer.writerow(temp)
    df = pd.DataFrame(pd.read_csv(r"gatesTemp.csv"))
    return df

In [11]:
def unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)


In [12]:
def depthFinder(df):
    qubits = df['Qubit'].value_counts()
    qubits = dict(qubits)
    lst = list(qubits.items())
    #get depth
    if len(lst) > 1:
        qnum = []
        for j in range(len(lst)):
            strList = str(lst[j][0]).split(',')
            qnum.append(strList[0])
        qnum = unique(qnum)
        counts = []
        for num in qnum:
            for j in range(len(lst)):
                strList = str(lst[j][0]).split(',')
                if strList[0] == num:
                    for times in range(lst[j][1]):
                        counts.append(strList[0])
        return len(counts) 
    elif len(lst) > 0:
        return(lst[0][1])
    else:
        return(0)
    

In [13]:
 def binary_search(cost, name):  
    low = 0  
    high = int((np.math.factorial(4 + name - 1))/(np.math.factorial(name)*(np.math.factorial(3)))) - 1
    mid = 0  
  
    while low <= high:  
        # for get integer result   
        mid = (high + low) // 2  
        # print("Name", name)
        try:
            dfFull = pd.read_csv("D:/Documents/Combinations/combo" + str(name) + ".csv", skiprows = mid, nrows=1)
        except:
            with open(r"ErrorLog.csv", 'w', newline='', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow([name])
                return -1
        # Check if n is present at mid   
        if dfFull.loc[0][0] < cost:  
            low = mid + 1  
  
        # If n is greater, compare to the right of mid   
    
        elif dfFull.loc[0][0] > cost:  
            high = mid - 1  
  
        # If n is smaller, compared to the left of mid  
        else:  
            return mid  
  
            # element was not present in the list, return -1  
    return -1  
# binary_search(25, 20)

In [14]:
def Average(lst):
    return sum(lst) / len(lst)

# Generate List of all Gate Combos

In [15]:
#No need to include phase gates, will replace x gates randomly with them
Gates2 = ['x', 'h', 'cx', 'swap']
maxComboLength = 290
bar = progressbar.ProgressBar(maxval=maxComboLength).start()
combs = []
for i in range(1, maxComboLength):
    #Files become too large to store on C drive. On external drive instead
    path = pathlib.Path("D:/Documents/Combinations/combo" + str(i) + ".csv")
    if path.exists():
        # print(i)
        pass
    else:
        with open(r"D:/Documents/Combinations/combo" + str(i) + ".csv", 'w', newline='', encoding='UTF8') as f:
            # create the csv writer
            writer = csv.writer(f)
            #Create header
            header = []
            header.append("Cost")
            for num in range(0, i):
                header.append(num)
            writer.writerow(header)
            for combination in itertools.combinations_with_replacement(Gates2, i):
                total = 0
                for g in combination:
                    total += gateCosts[Gates.index(g)]
                combination = list(combination)
                # print(combination)
                combination.insert(0, total)
                writer.writerow(combination)
        df = pd.read_csv("D:/Documents/Combinations/combo" + str(i) + ".csv")
        sorted_df = df.sort_values(by=["Cost"], ascending=True)
        sorted_df.to_csv("D:/Documents/Combinations/combo" + str(i) + ".csv", index=False)
    bar.update(i)

In [16]:
#Check if all files can be read, no corrupt files
for name in range(1, maxComboLength):
    clear_output(wait=True)
    print("Name", name)
    dfFull = pd.read_csv("D:/Documents/Combinations/combo" + str(name) + ".csv", skiprows = 0, nrows=1)

Name 289


## Circuit Generation

In [18]:
final = []
import time
bar = progressbar.ProgressBar(maxval=100).start()

#Error Log
with open(r"ErrorLog.csv", 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(["Error"])
            
#Cost has to be less than maxComboLength
#Can change cost in intervals to decrease time
for cost in range(1, 100 + 1):
    #find all combinations of gates to = cost, include entanglement as soon as cost reaches large enough number
    #max depth == cost
    if cost >= maxComboLength:
        break
    for totalGates in range(1, cost + 1):
        idx = binary_search(cost, totalGates)
        # print("idx", idx)
        # print("Cost:", cost)
        # print("Total Number of Gates:", totalGates)
        #No possible combination
        if idx != -1:
            combination = []
            df = pd.read_csv(r"D:/Documents/Combinations/combo" + str(totalGates) + ".csv", skiprows = idx, nrows=1)
            for i in df.loc[0]:
                if i in Gates:
                    combination.append(i)
            TT3 = []
            TT2 = []
            TT1 = []
            TT0 = []
            RT3 = []
            RT2 = []
            RT1 = []
            RT0 = []
            GC3 = []
            GC2 = []
            GC1 = []
            GC0 = []
            # for permutation in range(0, 30):
            clear_output(wait=True)
            print("Cost:", cost)
            print("Total Number of Gates:", totalGates)
            qc = QuantumCircuit(qubitCount)
            if combination != []:
                comboTemp = combination
                random.shuffle(comboTemp)
                combo = []
                #Replace all x gates with x, y, or z
                for gate in comboTemp:
                    if gate == 'x':
                        combo.append(Gates[random.randint(0,2)])
                    else:
                        combo.append(gate)
                #append all to first qubit until depth is met
                #the rest are distributed randomly to the others 
                for gateIdx in range(len(combo)):
                    if combo[gateIdx] == 'cx' or combo[gateIdx] == 'swap':
                        randint1 = random.randint(0, qubitCount - 1)
                        randint2 = random.randint(0, qubitCount - 1)
                        while randint2 == randint1:
                            randint2 = random.randint(0, qubitCount - 1)
                        getattr(qc, combo[gateIdx])(randint1, randint2)
                    else:
                        randint1 = random.randint(0, qubitCount - 1)
                        getattr(qc, combo[gateIdx])(randint1)
                # listToAppend.append(cost)
                # listToAppend.append(totalGates)
                qc.measure_all()
                print(qc)
                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 3)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                TT3.append(Average(tempTT))
                RT3.append(Average(tempRT))
                # listToAppend.append(result.get_counts())
                GC3.append(Average(tempD))

                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 2)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                TT2.append(Average(tempTT))
                RT2.append(Average(tempRT))
                # listToAppend.append(result.get_counts())
                GC2.append(Average(tempD))


                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 1)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                TT1.append(Average(tempTT))
                RT1.append(Average(tempRT))
                # listToAppend.append(result.get_counts())
                GC1.append(Average(tempD))

                tempRT = []
                tempTT = []
                tempD = []
                for run in range(runs):
                    start_time = time.time()
                    tqc =  transpile(qc, backend, optimization_level = 0)
                    tempTT.append(time.time() - start_time)
                    df = QCtoDF(tqc)
                    result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
                    tempRT.append(result.time_taken)
                    tempD.append(depthFinder(df))
                TT0.append(Average(tempTT))
                RT0.append(Average(tempRT))
                # listToAppend.append(result.get_counts())
                GC0.append(Average(tempD))



                # tqcSim = transpile(qc, backend, optimization_level = 3)
                # result = execute(tqcSim, Aer.get_backend('qasm_simulator')).result()
                # listToAppend.append(result.get_counts())
                TT3 = Average(TT3)
                TT2 = Average(TT2)
                TT1 = Average(TT1)
                TT0 = Average(TT0)
                RT3 = Average(RT3)
                RT2 = Average(RT2)
                RT1 = Average(RT1)
                RT0 = Average(RT0)
                GC3 = Average(GC3)
                GC2 = Average(GC2)
                GC1 = Average(GC1)
                GC0 = Average(GC0)
                # ['cost', 'gate count', 'tqc3TT', 'tqc3RT', 'tqc3GC', 'tqc2TT', 'tqc2RT', 'tqc2GC', 'tqc1TT', 'tqc1RT', 'tqc1GC', 'tqc0TT', 'tqc0RT', 'tqc0GC']
                listToAppend = [cost, totalGates, TT3, RT3, GC3, TT2, RT2, GC2, TT1, RT1, GC1, TT0, RT0, GC0]
                final.append(listToAppend)
        bar.update(cost)

Cost: 16
Total Number of Gates: 10
        ┌───┐           ░ ┌─┐                  
   q_0: ┤ H ├───────────░─┤M├──────────────────
        ├───┤┌───┐      ░ └╥┘┌─┐               
   q_1: ┤ H ├┤ H ├──────░──╫─┤M├───────────────
        └───┘└───┘      ░  ║ └╥┘┌─┐            
   q_2: ────────────────░──╫──╫─┤M├────────────
        ┌───┐┌───┐      ░  ║  ║ └╥┘┌─┐         
   q_3: ┤ X ├┤ H ├──────░──╫──╫──╫─┤M├─────────
        ├───┤├───┤┌───┐ ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ┤ H ├┤ X ├┤ Y ├─░──╫──╫──╫──╫─┤M├──────
        ├───┤├───┤└───┘ ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ┤ Z ├┤ H ├──────░──╫──╫──╫──╫──╫─┤M├───
        └───┘└───┘      ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ────────────────░──╫──╫──╫──╫──╫──╫─┤M├
                        ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/═══════════════════╩══╩══╩══╩══╩══╩══╩═
                           0  1  2  3  4  5  6 



KeyboardInterrupt



In [15]:
with open(r"QMachineData/PerthDataGC1.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(fields)
    writer.writerows(final)

### Test different mock backends

In [6]:
from qiskit import *
import pandas as pd
from qiskit import transpile
import random
import csv
import numpy as np
from qiskit.providers.aer import AerSimulator
import time
import itertools
import random
import progressbar
import pathlib

qubitCount = 16

from qiskit import IBMQ, Aer
from qiskit.providers.fake_provider import FakeGuadalupe
from qiskit.providers.aer.noise import NoiseModel
backend = FakeGuadalupe()
# backend = provider.get_backend('ibm_nairobi')
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

Gates = ['x', 'y', 'z', 'h', 'cx', 'swap']
gateCosts = [1, 1, 1, 2, 5, 11]
fields = ['cost', 'depth', 'tqc3TT', 'tqc3RT', 'tqc3Counts', 'tqc3Depth', 'tqc3Qubits', 'tqc2TT', 'tqc2RT', 'tqc2Counts', 'tqc2Depth', 'tqc2Qubits', 'tqc1TT', 'tqc1RT', 'tqc1Counts', 'tqc1Depth', 'tqc1Qubits', 'tqc0TT', 'tqc0RT', 'tqc0Counts', 'tqc0Depth', 'tqc0Qubits', 'sim']


In [12]:
from qiskit.tools.jupyter import *

In [13]:
backend

<FakeGuadalupe('fake_guadalupe')>

In [11]:
qc = QuantumCircuit(16)
qc.x(0)
qc.x(1)
qc.x(2)

qc.measure_all()
qc.draw()

┌───┐ ░ ┌─┐                                             
    q_0: ┤ X ├─░─┤M├─────────────────────────────────────────────
         ├───┤ ░ └╥┘┌─┐                                          
    q_1: ┤ X ├─░──╫─┤M├──────────────────────────────────────────
         ├───┤ ░  ║ └╥┘┌─┐                                       
    q_2: ┤ X ├─░──╫──╫─┤M├───────────────────────────────────────
         └───┘ ░  ║  ║ └╥┘┌─┐                                    
    q_3: ──────░──╫──╫──╫─┤M├────────────────────────────────────
               ░  ║  ║  ║ └╥┘┌─┐                                 
    q_4: ──────░──╫──╫──╫──╫─┤M├─────────────────────────────────
               ░  ║  ║  ║  ║ └╥┘┌─┐                              
    q_5: ──────░──╫──╫──╫──╫──╫─┤M├──────────────────────────────
               ░  ║  ║  ║  ║  ║ └╥┘┌─┐                           
    q_6: ──────░──╫──╫──╫──╫──╫──╫─┤M├───────────────────────────
               ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                        
    q_7: ──────░──╫──╫──╫──╫──╫──╫──╫─┤M├────────────────────────
               ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                     
    q_8: ──────░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────────────────
               ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                  
    q_9: ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────────────────
               ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐               
   q_10: ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───────────────
               ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐            
   q_11: ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────────────
               ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
   q_12: ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
               ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
   q_13: ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
               ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
   q_14: ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
               ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_15: ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
               ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 16/═════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                  0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15

In [12]:
t_qc = transpile(qc, backend, optimization_level =0)
t_qc.draw()

┌───┐ ░ ┌─┐                                             
  q_0 -> 0 ┤ X ├─░─┤M├─────────────────────────────────────────────
           ├───┤ ░ └╥┘┌─┐                                          
  q_1 -> 1 ┤ X ├─░──╫─┤M├──────────────────────────────────────────
           ├───┤ ░  ║ └╥┘┌─┐                                       
  q_2 -> 2 ┤ X ├─░──╫──╫─┤M├───────────────────────────────────────
           └───┘ ░  ║  ║ └╥┘┌─┐                                    
  q_3 -> 3 ──────░──╫──╫──╫─┤M├────────────────────────────────────
                 ░  ║  ║  ║ └╥┘┌─┐                                 
  q_4 -> 4 ──────░──╫──╫──╫──╫─┤M├─────────────────────────────────
                 ░  ║  ║  ║  ║ └╥┘┌─┐                              
  q_5 -> 5 ──────░──╫──╫──╫──╫──╫─┤M├──────────────────────────────
                 ░  ║  ║  ║  ║  ║ └╥┘┌─┐                           
  q_6 -> 6 ──────░──╫──╫──╫──╫──╫──╫─┤M├───────────────────────────
                 ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                        
  q_7 -> 7 ──────░──╫──╫──╫──╫──╫──╫──╫─┤M├────────────────────────
                 ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                     
  q_8 -> 8 ──────░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────────────────
                 ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                  
  q_9 -> 9 ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────────────────
                 ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐               
q_10 -> 10 ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───────────────
                 ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐            
q_11 -> 11 ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────────────
                 ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
q_12 -> 12 ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
                 ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
q_13 -> 13 ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
                 ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
q_14 -> 14 ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
                 ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q_15 -> 15 ──────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
                 ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
  meas: 16/═════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                    0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15

In [ ]:
result = execute(t_qc, backend,
                 coupling_map=coupling_map,
                 basis_gates=basis_gates,
                 noise_model=noise_model).result()

In [34]:
result.get_counts()

{'00101': 30,
 '01111': 17,
 '00110': 70,
 '10111': 28,
 '10110': 2,
 '00111': 859,
 '00100': 1,
 '01101': 1,
 '00011': 14,
 '00010': 1,
 '00001': 1}